In [13]:
import pandas as pd
from sqlalchemy import create_engine

## Extracting all the csv files as Pandas Dataframes

In [14]:
def extract_data():
    """Extract data from CSV files."""
    steps_df = pd.read_csv('downloads/Data/steps.csv')
    exercises_df = pd.read_csv('downloads/Data/exercises.csv')
    patients_df = pd.read_csv('downloads/Data/patients.csv')
    return steps_df, exercises_df, patients_df


In [15]:
steps_df, exercises_df, patients_df = extract_data()
steps_df.head()

,ID,EXTERNAL_ID,STEPS,SUBMISSION_TIME,UPDATED_AT
0,7075798,2227,163,2024-02-29T01:00:00.000+0100,2024-03-01T18:00:01.005+0100
1,7026711,2227,104,2024-02-23T01:00:00.000+0100,2024-02-23T06:00:01.013+0100
2,7033100,2227,4250,2024-02-23T01:00:00.000+0100,2024-02-24T06:00:01.197+0100
3,7038880,2227,36047,2024-02-24T01:00:00.000+0100,2024-02-25T04:30:01.313+0100
4,7035008,2227,38,2024-02-24T01:00:00.000+0100,2024-02-24T16:00:01.068+0100


In [16]:
exercises_df.head()

,ID,EXTERNAL_ID,MINUTES,COMPLETED_AT,UPDATED_AT
0,1760588,2227,1,2019-11-26T13:13:43.825+0100,2023-12-02T16:08:16.809+0100
1,25820128,2227,20,2023-07-21T16:27:15.000+0200,2024-01-03T15:29:01.871+0100
2,28660798,2227,2,2023-09-29T09:19:10.345+0200,2023-12-02T16:08:16.809+0100
3,12678534,2227,3,2022-06-09T14:32:54.000+0200,2023-12-02T16:08:16.809+0100
4,2383166,2227,1,2020-04-22T17:00:37.181+0200,2023-12-02T14:28:01.591+0100


In [17]:
patients_df.head()

,Unnamed: 0,PATIENT_ID,first_name,last_name,country
0,0,2227,Emily,Castaneda,Ethiopia
1,1,5744,Logan,Coleman,Romania
2,2,6068,David,Dickerson,Netherlands Antilles
3,3,7789,Lance,Miranda,Yemen
4,4,8090,Heather,Moore,Niger


## Get a brief summary of all the dataframes

In [18]:
steps_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11909 entries, 0 to 11908
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               11909 non-null  int64 
 1   EXTERNAL_ID      11909 non-null  int64 
 2   STEPS            11909 non-null  int64 
 3   SUBMISSION_TIME  11909 non-null  object
 4   UPDATED_AT       11909 non-null  object
dtypes: int64(3), object(2)
memory usage: 465.3+ KB


In [19]:
exercises_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65427 entries, 0 to 65426
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            65427 non-null  int64 
 1   EXTERNAL_ID   65427 non-null  int64 
 2   MINUTES       65427 non-null  int64 
 3   COMPLETED_AT  62596 non-null  object
 4   UPDATED_AT    65427 non-null  object
dtypes: int64(3), object(2)
memory usage: 2.5+ MB


In [20]:
patients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  100 non-null    int64 
 1   PATIENT_ID  100 non-null    int64 
 2   first_name  100 non-null    object
 3   last_name   100 non-null    object
 4   country     100 non-null    object
dtypes: int64(2), object(3)
memory usage: 4.0+ KB


## Data Transformation to calculate the total minutes of wach patient

In [21]:
def transform_data(steps_df, exercises_df, patients_df):
    """Transform data to calculate total minutes for each patient."""
    # Create SQLAlchemy engine
    engine = create_engine('sqlite:///:memory:')

    # Load DataFrames into SQLite database
    steps_df.to_sql('steps', con=engine, if_exists='replace', index=False)
    exercises_df.to_sql('exercises', con=engine, if_exists='replace', index=False)
    patients_df.to_sql('patients', con=engine, if_exists='replace', index=False)

    # Transform: Calculate total minutes for each patient using SQL
    query = """
            SELECT p.Patient_id,
                   p.First_name,
                   p.Last_name,
                   p.Country,
                   COALESCE(SUM(s.Steps) * 0.002, 0) + COALESCE(SUM(e.Minutes), 0) AS total_minutes
            FROM patients p
            LEFT JOIN (
                SELECT External_id, SUM(Steps) AS Steps
                FROM steps
                GROUP BY External_id
            ) s ON p.Patient_id = s.External_id
            LEFT JOIN (
                SELECT External_id, SUM(Minutes) AS Minutes
                FROM exercises
                GROUP BY External_id
            ) e ON p.Patient_id = e.External_id
            GROUP BY p.Patient_id
            ORDER BY total_minutes DESC;
            """
    result = engine.execute(query)
    result_df = pd.DataFrame(result, columns=['patient_id', 'first_name', 'last_name', 'country', 'total_minutes'])
    return result_df


In [22]:
result_df = transform_data(steps_df, exercises_df, patients_df)
result_df.head()

,patient_id,first_name,last_name,country,total_minutes
0,356134,Austin,Ellis,Germany,110592.540
1,269286,Morgan,Kirby,Antarctica (the territory South of 60 deg S),45075.418
2,298048,Mark,Anderson,Aruba,25404.384
3,11063,Lisa,Cruz,Papua New Guinea,24609.974
4,434057,Willie,Nolan,Saint Vincent and the Grenadines,21775.266


In [23]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   patient_id     100 non-null    int64  
 1   first_name     100 non-null    object 
 2   last_name      100 non-null    object 
 3   country        100 non-null    object 
 4   total_minutes  100 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.0+ KB


In [24]:
def load_data(result_df):
    """Load transformed data to a CSV file."""
    result_df.to_csv('downloads/output/highest_minutes_patients.csv', index=False)
    print("Data saved to downloads/output/highest_minutes_patients.csv")

## ETL Process

In [25]:
def main():
    """Main function to execute the ELT pipeline."""
    # Extract data
    steps_df, exercises_df, patients_df = extract_data()
    
    # Transform data
    result_df = transform_data(steps_df, exercises_df, patients_df)
    
    # Load data
    load_data(result_df)

if __name__ == "__main__":
    main()

Data saved to downloads/output/highest_minutes_patients.csv
